<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
 

<strong>Objectives:</strong>

    - Estudiar y explorar las series correspondientes al producto 68
    - Comprobar las estacionalidades
    - Descomponer la serie
    - Obtener la media por día de la semana, mes, etc.
    - Estudiar la asignación de missings en ventas y stock
    
    
<strong>Conclusions:</strong>

   
</div>

In [1]:
import os, sys

def get_root_path(search_word):
    """ Búsqueda hacia atrás de un directorio con nombre search_word. """
    cwd = os.getcwd()
    while str(os.getcwd()).split('/')[-1] != search_word:
        os.chdir("..")
        cwd = os.getcwd()
    return cwd

ROOT_DIR = get_root_path('TFM_StockForecast')

sys.path.insert(0, ROOT_DIR) # Poner en el primer lugar del PATH (evitar paquetes con mismos nombres en otros directorios)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.load_data import load_csv, load_data
from src.prepare_data import prepare_train_data

In [2]:
from statsmodels.tsa.seasonal import seasonal_decompose

def plot_seasonal_descompose(serie, period):
    
    result = seasonal_decompose(serie, model='additive', period=period)
    
    plt.figure(figsize=(20,40))
    plt.subplot(4,1,1)
    result.observed.plot(title='Observed')
    plt.subplot(4,1,2)
    result.trend.plot(title='Trend')
    plt.subplot(4,1,3)
    result.seasonal.plot(title='Seasonal - Period {}'.format(period))
    plt.subplot(4,1,4)
    result.resid.plot(title='Residual')
    plt.tight_layout()

In [3]:
df = load_data()
df.head()

===================  LOAD DATASET stock ====================
Reading CSV in data/raw/03_TablaStock.csv...
=============  CLEAN CSV  ==============
Set columns to lowcase.
Sort values by date.
Dropped duplicates. Rows dropped: 628.
============  CLEAN STOCK  =============
Dropped rows with data from products with no stock data or out-of-date (descatalogados). Rows dropped: 1784
[WARNING] Number of dates missings:                                83
[WARNING] Number of dates missings (droping sundays and festivos): 63
[WARNING] Stock data with different units for same product & data. Rows: 12666
Dataset stock loaded. Shape: (25518, 3)

===================  LOAD DATASET ventas ===================
Reading CSV in data/raw/01_TablaVentas.csv...
=============  CLEAN CSV  ==============
Set columns to lowcase.
Sort values by date.
Dropped duplicates. Rows dropped: 0.
============  CLEAN VENTAS  ============
Dropped rows with data from products with no stock data or out-of-date (descatalogados). 

/mnt/c/Users/Administrador/Documents/GitHub/TFM_StockForecast/src/create_variables.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts['missing'] = (ts.udsstock.isna()) * 1


Getting datetime variables: weekday, quarter, month, weekofyear
Getting deltaStock as the difference of Stock from today to tomorrow


/mnt/c/Users/Administrador/Documents/GitHub/TFM_StockForecast/src/create_variables.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prod_data['deltaStock'] = prod_data.udsstock.diff(periods=-1)


Getting rolling windows of last 5 days by product and weekday for column udsventa


/mnt/c/Users/Administrador/Documents/GitHub/TFM_StockForecast/src/create_variables.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data[colname] = day_data[col].rolling(4, win_type='triang', min_periods=1).mean()
/mnt/c/Users/Administrador/Documents/GitHub/TFM_StockForecast/src/create_variables.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data["meanwd_"+col] = day_data[col].mean()


Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by product and weekday for column udsprevisionempresa


,fecha,producto,udsstock,udsventa,udsprevisionempresa,promo,festivo,stockMissingType,weekday,quarter,month,weekofyear,deltaStock,roll4wd_udsventa,meanwd_udsventa,roll4wd_udsstock,meanwd_udsstock,roll4wd_udsprevisionempresa,meanwd_udsprevisionempresa
0,2019-06-05,1,18275.0,2833.0,102304.0,0.0,0.0,0.0,2,2,6,23,NaN,2833.0,3405.131579,18275.0,11012.218750,102304.0,37273.000000
1,2019-06-05,10,2267.0,1874.0,36888.0,1.0,0.0,0.0,2,2,6,23,NaN,1874.0,2440.763158,2267.0,1822.161290,36888.0,10102.540541
2,2019-06-05,11,2506.0,2755.0,31078.0,0.0,0.0,0.0,2,2,6,23,NaN,2755.0,2323.135135,2506.0,3987.666667,31078.0,10973.815789
3,2019-06-05,12,1279.0,1161.0,33661.0,0.0,0.0,0.0,2,2,6,23,NaN,1161.0,1143.368421,1279.0,1605.741935,33661.0,5724.236842
4,2019-06-05,13,2493.0,1603.0,32119.0,1.0,0.0,0.0,2,2,6,23,NaN,1603.0,1874.447368,2493.0,3113.843750,32119.0,8500.421053


In [4]:
data = df.loc[(df.producto=='68')]
data.head()

,fecha,producto,udsstock,udsventa,udsprevisionempresa,promo,festivo,stockMissingType,weekday,quarter,month,weekofyear,deltaStock,roll4wd_udsventa,meanwd_udsventa,roll4wd_udsstock,meanwd_udsstock,roll4wd_udsprevisionempresa,meanwd_udsprevisionempresa
50,2019-06-05,68,1545.0,392.0,7451.0,0.0,0.0,0.0,2,2,6,23,NaN,392.0,428.589744,1545.0,1007.093750,7451.0,1753.947368
122,2019-06-06,68,NaN,428.0,9499.0,0.0,0.0,2.0,3,2,6,23,NaN,428.0,430.425000,NaN,964.343750,9499.0,2238.692308
194,2019-06-07,68,NaN,510.0,5106.0,0.0,0.0,2.0,4,2,6,23,NaN,510.0,506.641026,NaN,794.533333,5106.0,1444.564103
266,2019-06-08,68,NaN,81.0,1735.0,0.0,0.0,2.0,5,2,6,23,NaN,81.0,105.850000,NaN,1140.357143,1735.0,715.736842
338,2019-06-09,68,NaN,NaN,NaN,0.0,0.0,2.0,6,2,6,23,NaN,NaN,32.500000,NaN,918.208333,NaN,NaN


In [5]:
data = prepare_train_data(data)

data.head()

===================  FILTER TRAIN DATA  ====================
Dropped rows corresponding to 23 to 26-03-2020 for not having the ventas data for these days.
Rows dropped: 4
Assigning missings for udsstock
Missing in dataset:               85 (292 total rows).
Missings in holiday days:         20 (50 total rows).
Missings in working days:         65 (242 total rows).
Assigned missings for working days data - Remaining missings: 25
Assigned missings for holiday data - Remaining missings:      5
Assigned left missings with the mean of the weekday - Remaining missings: 0
Assigned missings in udsventa.
Assigned missings in udsprevisionempresa filling with 0
Output shape: (292, 20)


,fecha,producto,udsstock,udsventa,udsprevisionempresa,promo,festivo,stockMissingType,weekday,quarter,month,weekofyear,deltaStock,roll4wd_udsventa,meanwd_udsventa,roll4wd_udsstock,meanwd_udsstock,roll4wd_udsprevisionempresa,meanwd_udsprevisionempresa,holiday_udsstock
0,2019-06-05,68,1545.000000,392.0,7451.0,0.0,0.0,0.0,2,2,6,23,NaN,392.0,428.589744,1545.0,1007.093750,7451.0,1753.947368,1545.0
1,2019-06-06,68,964.343750,428.0,9499.0,0.0,0.0,2.0,3,2,6,23,NaN,428.0,430.425000,NaN,964.343750,9499.0,2238.692308,1429.0
2,2019-06-07,68,794.533333,510.0,5106.0,0.0,0.0,2.0,4,2,6,23,NaN,510.0,506.641026,NaN,794.533333,5106.0,1444.564103,1429.0
3,2019-06-08,68,1140.357143,81.0,1735.0,0.0,0.0,2.0,5,2,6,23,NaN,81.0,105.850000,NaN,1140.357143,1735.0,715.736842,1429.0
4,2019-06-09,68,1429.000000,0.0,0.0,0.0,0.0,2.0,6,2,6,23,NaN,NaN,32.500000,NaN,918.208333,NaN,NaN,1429.0


In [6]:
data.loc[data.udsstock.isna()]

,fecha,producto,udsstock,udsventa,udsprevisionempresa,promo,festivo,stockMissingType,weekday,quarter,month,weekofyear,deltaStock,roll4wd_udsventa,meanwd_udsventa,roll4wd_udsstock,meanwd_udsstock,roll4wd_udsprevisionempresa,meanwd_udsprevisionempresa,holiday_udsstock


In [ ]:
data = data[["udsventa","udsprevisionempresa", "udsstock"]]

data.info()

In [ ]:
actual = np.array([1,2,3,1])
err_naive = np.mean(np.diff(np.array(actual)))

In [ ]:
actual = np.array([1,2,3,1])
np.mean(np.abs(np.diff(np.array(actual))))

# Time Series

In [ ]:
subdf = df.loc[(df.producto=='68')]
for series in ['udsventa', 'udsstock', 'udsprevisionempresa']:
    subdf[[series, 'roll4wd_'+series]].plot(figsize=(20,10), title="Producto: {}".format('68'))
    plt.show()
    


In [ ]:
def _assing_missings_stock(df, col):
    """Assign missings: backfill for holiday
                        4wd rolling windows for else???
    """
    print("Assigning missings for {}".format(col))
    data = df.copy()
    holiday_data = data.loc[(data['festivo'] == 1) | (data['weekday'] == 6)]
    wd_data = data.loc[(data['festivo'] != 1) & (data['weekday'] != 6)]
    
    print("Missing in dataset:               {} ({} total rows).".format(sum(data[col].isna()), data.shape[0]))
    print("Missings in holiday days:         {} ({} total rows).".format(sum(holiday_data[col].isna()), holiday_data.shape[0]))
    print("Missings in working days:         {} ({} total rows).".format(sum(wd_data[col].isna()), wd_data.shape[0]))
    
    data['holiday_' + col] = data.fillna(method='bfill')
    print("Assigned missings for holiday data - Remaining missings:      {}".format(sum(data[col].isna())))
    
    data.loc[(data['festivo'] != 1) & (data['weekday'] != 6), col] = wd_data[col].fillna(wd_data["roll4wd_" + col])
    print("Assigned missings for working days data - Remaining missings: {}".format(sum(data[col].isna())))
    return data
col = 'udsstock'
data = subdf.copy()

# Creamos vectores de filtro
holiday = (data['festivo'] == 1) | (data['weekday'] == 6)
workday = (data['festivo'] != 1) & (data['weekday'] != 6)
missing = data[col].isna()
incrementnot0 = (~data['deltaStock'].isna()) & (data['deltaStock'] != 0)


print("Missing in dataset:               {} ({} total rows).".format(sum(missing), data.shape[0]))
print("Missings in holiday days:         {} ({} total rows).".format(sum(missing & holiday), sum(holiday)))
print("Missings in working days:         {} ({} total rows).".format(sum(missing & workday), sum(workday)))

print("In holiday days, the increment of stock was different from 0 in {} days.".format(sum(holiday & incrementnot0)))

# Creamos una columna para los holiday missings
data['holiday_missing'] = data[col].fillna(method='bfill')

# Asignamos todos los valores como rolling windows
data[col] = data[col].fillna(data["roll4wd_" + col])
print("Assigned missings for working days data - Rolling windows method.")

# Asignamos los holidays por encima y borramos la columna intermedia
data.loc[holiday & missing, col] = data.loc[holiday & missing, "holiday_missing"]
data = data.drop("holiday_missing", axis=1)
print("Assigned missings for holiday data - backfill method")
print("Assigned missings for holiday data - Remaining missings: {}".format(sum(data[col].isna())))



data


In [ ]:
subdf = df.loc[(df.producto=='1')]
subdf['deltaStock1'] = subdf.udsstock.diff(periods=1)
subdf.loc[subdf.weekday == 6]

In [ ]:
plot_seasonal_descompose(subdf.udsventa, 7)

## Missings de ventas

In [ ]:
miss[['weekday','udsventa']].fillna(0).groupby("weekday")['udsventa'].count()

In [ ]:
data = subdf
col = 'roll5wd_udsventa'
miss = data.loc[data[col].isna()]
no_miss = data.loc[~data[col].isna()]
# Numero total de missings
print("Missings en la serie:                     {}".format(miss.shape[0]))
print("Missings en la serie sin cambio de stock: {}".format(miss.loc[miss.deltaStock == 0].shape[0]))
print("Missings en la serie con cambio de stock: {}".format(miss.loc[miss.deltaStock > 0].shape[0]))
print("Missings en la serie con cambio de stock: {}".format(miss.loc[miss.deltaStock < 0].shape[0]))


# Asignamos 

In [ ]:
data = subdf
col = 'udsventa'
miss = data.loc[data[col].isna()]
no_miss = data.loc[~data[col].isna()]
# Numero total de missings
print("Missings en la serie:                     {}".format(miss.shape[0]))
print("Missings en la serie sin cambio de stock: {}".format(miss.loc[miss.deltaStock == 0].shape[0]))
print("Missings en la serie con cambio de stock: {}".format(miss.loc[miss.deltaStock > 0].shape[0]))
print("Missings en la serie con cambio de stock: {}".format(miss.loc[miss.deltaStock < 0].shape[0]))

In [ ]:
no_miss.loc[no_miss.deltaStock == 0]

In [ ]:
miss.loc[miss.deltaStock == 0]

In [ ]:
subdf.loc[subdf.weekday == 6]

In [ ]:
subdf.loc[subdf.index > '2019-08-12']

## Medias de las ventas

In [ ]:
# Media de ventas por día de la semana
subdf.groupby("weekday")['udsventa'].mean()

In [ ]:
# Media de ventas por día de la semana
subdf.groupby("weekday")['udsventa'].min()

In [ ]:
# Media de ventas por día de la semana
subdf.groupby("weekday")['udsventa'].max()

In [ ]:
# Media de ventas por día de la semana
subdf.groupby("festivo")['udsventa'].min()

In [ ]:
# Media de ventas por día de la semana
subdf.groupby("festivo")['udsventa'].max()

In [ ]:
subdf

## Ventas en festivo o domingo faltantes puede ser 0?

In [ ]:
subdf = df.loc[(df.producto=='68')]
subdf.udsventa.plot(figsize=(20,10), title="Producto: {}".format('68'))
plt.show()

In [ ]:
subdf.loc[(subdf.festivo == 1) | (subdf.weekday == 6)] = subdf.loc[(subdf.festivo == 1) | (subdf.weekday == 6)].fillna(0)
subdf.udsventa.plot(figsize=(20,10), title="Producto: {}".format('68'))
plt.show()

In [ ]:
subdf.loc[(subdf.festivo == 1) | (subdf.weekday == 6)] = subdf.loc[(subdf.festivo == 1) | (subdf.weekday == 6)].fillna(0)
subdf.udsventa.plot(figsize=(20,10), title="Producto: {}".format('68'))
plt.show()

In [ ]:
subdf.loc[subdf.udsventa.isna()]